# La Types génériques

---

> Les types génériques sont une des composantes fondamentales du langage Rust, ils permettent d'utiliser les mêmes traits, méthodes ou fonction pour plusieurs types différents sans avoir à définir des implémentations pour chacun de ces types. On utilise une lettre majuscule pour nommer un type générique, la plus fréquemment utilisée est la lettre `T`.

## La STD et les génériques

> La standard librairy de Rust définit énormément de structures, d'énumérations ou de traits qui prennent en paramètre des types génériques. Nous avons déjà croisé les énumérations `Option<T>` ou les `Result<T, E>`. Il y a aussi les structures dynamiquement allouées tel que les `Box<T, A = Global>` ou les `Vec<T, A = Global>`, ou des traits comme `From<T>` et `Into<T>` ou encore des fonctions comme `parse<F>`...

> Toutes ces lettres en majuscule expriment des types génériques, que le moteur d'inférence tachera d'inférer correctement ou bien que nous devrons spécifier à la main.

### Utilisation transparente des types génériques

- Souvent, le moteur d'inférence vient en aide au développeur afin de lui épargner la tache de complexifier la syntaxe de son code, en voici des exemples :

In [48]:
let u: usize = 12;
let opt = Some(u); // Ici le type implicite de opt est Option<usize>
    
let my_box = Box::new(u); // Le type implicite est Box<usize>
    
let my_integer: isize = "1664".parse().unwrap(); // Le T de la fonction parse() est infere en usize ...
                                                 // ... determine par le type de sortie

### L'annotation du type

- Mais il arrive que nous ayons à renseigner Rust sur le type à utiliser :

In [47]:
{
    let my_integer = "1664".parse().unwrap();
}

Error: type annotations needed

> Dans ce cas, nous avons deux solutions :
> - **Donner le type de la variable result comme dans l'exemple précèdent et compter sur l'inférence.**
> - Utiliser un **turbofish** pour expliciter l'annotation du type.

Un turbofish ressemble à un petit poisson et sa syntaxe est de la forme `::<__type__>` soit `::<isize>` ici. Il ne faut pas oublier les deux petits `::` avant le symbole `<`.

**Mais ou mettre le turbofish ?** (cf: Where to put the turbofish ?)

Regardons le prototype de la fonction parse :
```
pub fn parse<F>(&self) -> Result<F, <F as FromStr>::Err>
where
    F: FromStr,
```
- Ici, la lettre F est placée entre le nom de la fonction et la parenthèse ouvrante des paramètres. C'est donc à cet endroit que doit se placer le turbofish `::<isize>`:

In [49]:
{
    let my_integer = "1664".parse::<isize>().unwrap(); // Le turbofish est a sa place !
    dbg!(my_integer);
}

[src/lib.rs:134] my_integer = 1664


()

> parse() est une méthode ou une fonction.

- Le placement du turbofish sur un trait est un peu différent, prenons l'exemple du trait Into :
```
pub trait Into<T>: Sized {
    // Required method
    fn into(self) -> T;
}
```
- La déclaration du type générique T est juste après le nom du Trait `Into` + `<T>`, Le turbofish apparaîtra donc a cet endroit :

In [3]:
// let s = "Banane".into(); // Ne compile pas sans declaration du type ni turbofish
let s = Into::<String>::into("Banane"); // Ici, le turbofish renseigne correctement le compilateur
dbg!(s);

[src/lib.rs:105] s = "Banane"


**IMPORTANT**

> Pour Into ici, on utilise la syntaxe `Trait::methode` avec les `::` entre le trait et la méthode. Avec l'ajout du turbofish pour le trait on a donc `Trait::<type>::methode`. Cette syntaxe avec les `::` est équivalente à celle que l'on aurait pour un constructeur de structure `Structure::new`. `Trait` ou `Structure` ici constituent un namespace.

> Pour une instruction comme `let a: usize = 42 + 42;`, on aurait pu écrire `let a = std::ops::Add::<usize>::add(42, 42);` qui est sa version sans aucun sucre syntaxique.

## Type générique dans une structure ou une énumération

> Dans cette partie, nous allons écrire la définition d'une structure utilisant un type générique et créer une méthode générique pour cette dernière. Cet exemple, je l'espère, permettra déjà de se faire une bonne idée de ce que sont les types génériques.

### Ce que l'on avait déjà

- Reprenons la structure Vector qui a déjà été croisée plusieurs fois dans le cours :

In [5]:
{
    struct Vector {
        i: usize,
        j: usize,
    }
}

    L'inconvénient de cette structure, c'est qu'elle ne peut s'utiliser qu'avec le type usize, or on aimerait bien pouvoir utiliser des valeurs négatives ou des valeurs flottantes sans avoir à définir 3 types de structure !

### Écriture générique de la structure Vector

- Afin de rendre les types de ma structure génériques, je dois remplacer `usize` par `T`. Attention, comme T n'est pas déclaré pour ma structure, je dois mettre en suffixe du nom de ma structure la lettre `T` enveloppée de `<` et de `>`.

In [6]:
{
    struct Vector<T> {
        i: T,
        j: T,
    }
}

> Omettre `<T>` provoque une erreur de compilation.

### Déclaration d'une variable de type Vector

- Essayons de déclarer plusieurs Vector avec des valeurs aux allures différentes :

In [ ]:
{
    struct Vector<T> { i: T, j: T }
    let v1 = Vector { i: 32, j: 64 };
    let v2 = Vector { i: 3.14, j: 6.78 };
    let v3 = Vector { i: -2, j: -87};
}

- Les types génériques semblent fonctionner. Mais le moteur d'inférence de type vient de nous filer un sacre coup de main pour la syntaxe. En réalité, si nous avions eu à définir le type exact de Vector pour v1, v2 et v3, la syntaxe aurait été comme cela :

In [15]:
{
    struct Vector<T> { i: T, j: T }
    let v1: Vector<usize> = Vector { i: 32, j: 64 };
    let v2: Vector<f64> = Vector { i: 3.14, j: 6.78 };
    let v3: Vector<isize> = Vector { i: -2, j: -87};
}

()

> Après le nom du type de structure, la définition du type générique doit figurer entre `< >`

- Nous remarquons que c'est aussi possible de mettre un booléen en type, pourtant ça ne parait pas logique... pour un vecteur.

In [19]:
{
    struct Vector<T> { i: T, j: T }
    let v4: Vector<bool> = Vector { i: false, j: true };
}

()

    Il n'y a aucune contrainte à ce que ce soit tel ou tel type. Ils peuvent tous y passer !

### Les contraintes de trait

- Certains types ne peuvent pas s'additionner, ainsi, nous préférerions éviter de pouvoir renseigner ces types-là. Pour cela, nous allons contraindre la définition de la structure Vector en exigeant que le type `T` DOIT implémenter le trait Add :

In [21]:
{
    struct Vector<T: std::ops::Add> {
        i: T,
        j: T,
    }
}

()

La syntaxe correcte consiste à l'ajout du symbole `:` après la déclaration du type générique suivi d'un ou de plusieurs traits (utiliser `+` pour ca).

On pourrait avoir `struct Vector<T: std::ops::Add + std::ops::AddAssign + std::fmt::Debug>` si l'on veut que le type implémente Add, AddAssign et Debug.

In [31]:
{
    #[derive(Debug)]
    struct Vector<T: std::ops::Add + std::fmt::Debug> {
        i: T,
        j: T,
    }
    let v4: Vector<f64> = Vector { i: 3.13, j: -1.3 };
    dbg!(v4);
}

[src/lib.rs:33] v4 = Vector {
    i: 3.13,
    j: -1.3,
}


()

- Ça ne fonctionne plus avec le type bool puisque bool n'implémente pas Add :

In [30]:
{
    struct Vector<T: std::ops::Add + std::fmt::Debug> {
        i: T,
        j: T,
    }
    let v4: Vector<bool> = Vector { i: false, j: true };
}

Error: cannot add `bool` to `bool`

Error: cannot add `bool` to `bool`

### Implémentation de Vector\<T>

- Implémenter des méthodes pour la structure ne sera plus pour Vector, mais pour `Vector<T>`. Le type Vector tout court n'existe pas pour le compilateur. Dans la syntaxe de mon implémentation, je dois rappeler les différentes contraintes de trait pour Vector\<T> soit `std::ops::Add + std::fmt::Debug` pour notre exemple :

In [33]:
{
    #[derive(Debug)]
    struct Vector<T> { i: T, j: T}
    fn take_vector<T: std::ops::Add + std::fmt::Debug>(vector: Vector<T>) {}
    impl<T: std::ops::Add + std::fmt::Debug> Vector<T> { // Bien respected la syntaxe !
        // Implementation des methodes ici
    }
}

()

- Définissons une méthode debug(), cela se fait sans difficulté particuliere, la syntaxe `&self` des méthodes reste la même :

In [35]:
{
    #[derive(Debug)]
    struct Vector<T> { i: T, j: T}
    impl<T: std::ops::Add + std::fmt::Debug> Vector<T> { // Bien respecter la syntaxe !
        fn debug(&self) {
            dbg!(self); 
        }
    }
    let s = Vector { i: 3.12, j: 9.1 };
    s.debug();
}

[src/lib.rs:32] self = Vector {
    i: 3.12,
    j: 9.1,
}


()

### Fonctions prenant Vector\<T>

- Les fonctions prenant en paramètre Vector\<T> ou retournant Vector\<T> auront grosso modo la même syntaxe que l'implémentation. Exemple de fonction prenant Vector\<T> :

In [40]:
{
    #[derive(Debug)]
    struct Vector<T> { i: T, j: T}
    fn create_vector<T: std::ops::Add + std::fmt::Debug>(i: T, j: T) -> Vector<T> {
        Vector {
            i,
            j,
        }
    }
    fn take_vector<T: std::ops::Add + std::fmt::Debug>(vector: Vector<T>) { // Bien respecter la syntaxe !
        dbg!(vector);
    }
    let v = create_vector(3.12, 9.1);
    take_vector(v);
}

[src/lib.rs:36] vector = Vector {
    i: 3.12,
    j: 9.1,
}


()

### Implémentation du trait Add pour Vector\<T>

- Écrivons donc le boilerplate du trait Add pour notre vecteur générique :

In [7]:
{
    #[derive(Debug)]
    struct Vector<T: std::ops::Add + std::fmt::Debug> { i: T, j: T}
    impl <T: std::ops::Add<Output = T> + std::fmt::Debug>std::ops::Add for Vector<T> {
        type Output = Self;
        fn add(self, other: Self) -> Self {
            Vector {
                i: self.i + other.i,
                j: self.j + other.j,
            }
        }
    }
    dbg!(Vector { i: 3.14, j: 6.5} + Vector { i: 6.9, j: -1.12}); // Addition de vecteurs float
    dbg!(Vector { i: 3, j: 6} + Vector { i: 6, j: 1}); // Addition de vecteurs entiers positifs
    dbg!(Vector { i: -3, j: 6} + Vector { i: -6, j: -2}); // Addition de vecteurs entiers relatifs
}

[src/lib.rs:132] Vector { i: 3.14, j: 6.5 } + Vector { i: 6.9, j: -1.12 } = Vector {
    i: 10.040000000000001,
    j: 5.38,
}
[src/lib.rs:133] Vector { i: 3, j: 6 } + Vector { i: 6, j: 1 } = Vector {
    i: 9,
    j: 7,
}
[src/lib.rs:134] Vector { i: -3, j: 6 } + Vector { i: -6, j: -2 } = Vector {
    i: -9,
    j: 4,
}


()

> Ce code est velu pour un débutant ! On peut reconnaître le groupe `<T: std::ops::Add<Output = T> + std::fmt::Debug>` juste après le mot `impl` comme pour l'implémentation de Vector\<T> mais avec le sous groupe `<Output = T>` en plus pour Add...

```
Trait std::ops::Add

pub trait Add<Rhs = Self> {
    type Output;

    // Required method
    fn add(self, rhs: Rhs) -> Self::Output;
}
```
> En fait, lorsque l'on implémente le trait Add a la main, on doit définir Output (qui ne sera donc pas forcément du type de ce que l'on additionne). Ici, Add est une contrainte pour T, puisque que l'on utilise l'addition de deux types T avec en résultat un type T `self.i + other.i` et `self.j + other.j`, le compilateur a besoin de connaître le type que l'on attend en résultat...

**NB il est aussi possible d'implémenter un trait pour un type générique, il y a un exemple dans le chapitre 12.**